In [1]:
%idle_timeout 15
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2
%connections rdspostgresqlpoc



import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 15 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2
Connections to be included:
xledmpoc
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Idle Timeout: 15
Session ID: ad263024-fdde-4dda-9207-19c37ab774e2
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session ad263024-fdde-4dda-9207-19c37ab774e2 to get into ready status...
Session ad2630

In [6]:
# additional libraries
from pyspark.sql.functions import lit
import uuid
import random
from datetime import datetime, timedelta
from awsglue.dynamicframe import DynamicFrame

In [ ]:
def random_timestamp(start, end):
    return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

num_rows = 1000000  # fill amount of data want to generate
start_date = datetime(2024, 8, 1, 0, 0, 0)
end_date = datetime(2024, 8, 21, 23, 59, 59)
types = ["TypeA", "TypeB", "TypeC", "TypeD", "TypeE"]  

# i = 0
for _ in range(num_rows):
    if _%10000==0:
        
        # Generate data
        data = []
        for _ in range(10000):
            id_value = str(uuid.uuid4())
            inserted_at = random_timestamp(start_date, end_date)
            pubsub_message_id = f"MSG{random.randint(100, 999)}"
            amountRange1 = f"{random.randint(100, 500)}-{random.randint(600, 1000)}"
            amountRange2 = f"{random.randint(100, 500)}-{random.randint(600, 1000)}"
            type_value = random.choice(types)
            schg = f"{random.randint(50, 150)}-{random.randint(200, 300)}"
            validFrom = random_timestamp(start_date, end_date)

            data.append((id_value, inserted_at, pubsub_message_id, amountRange1, amountRange2, type_value, schg, validFrom))

        columns = ["id", "inserted_at", "pubsub_message_id", "amountRange1", "amountRange2", "type", "schg", "validFrom"]
        df = spark.createDataFrame(data, schema=columns)
        
        dy_s_chg = DynamicFrame.fromDF(df, glueContext, "dy_s_chg")
        glueContext.write_dynamic_frame.from_jdbc_conf(
                        frame = dy_s_chg, 
                        catalog_connection = "xledmpoc", 
                        connection_options = {
                            "dbtable": "s_chg",  # The target table in your RDS PostgreSQL
                            "database": "edmpoc"  # The target database in your RDS PostgreSQL
                        },
                        transformation_ctx = "s_chg"
                    )

       

In [3]:
# df = spark.read \
#     .format("parquet") \
#     .load("s3://xlaxiata-poc-edm-touchpoint/DummyData/s_chgs_dummy/")

In [3]:
# def random_timestamp(start, end):
#     return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

# num_rows = 1000000  # fill amount of data want to generate
# start_date = datetime(2024, 8, 1, 0, 0, 0)
# end_date = datetime(2024, 8, 21, 23, 59, 59)
# types = ["TypeA", "TypeB", "TypeC", "TypeD", "TypeE"]  

# i = 0
# for _ in range(num_rows):
#     if _%10000==0:
        
#         # Generate data
#         data = []
#         for _ in range(10000):
#             id_value = str(uuid.uuid4())
#             inserted_at = random_timestamp(start_date, end_date)
#             pubsub_message_id = f"MSG{random.randint(100, 999)}"
#             amountRange1 = f"{random.randint(100, 500)}-{random.randint(600, 1000)}"
#             amountRange2 = f"{random.randint(100, 500)}-{random.randint(600, 1000)}"
#             type_value = random.choice(types)
#             schg = f"{random.randint(50, 150)}-{random.randint(200, 300)}"
#             validFrom = random_timestamp(start_date, end_date)

#             data.append((id_value, inserted_at, pubsub_message_id, amountRange1, amountRange2, type_value, schg, validFrom))

#         columns = ["id", "inserted_at", "pubsub_message_id", "amountRange1", "amountRange2", "type", "schg", "validFrom"]
#         df = spark.createDataFrame(data, schema=columns)

#         nama_file = "s_chgs_dummy"
#         s3_bucket = f"s3://xlaxiata-poc-edm-touchpoint/DummyData/{nama_file}"
#         df.coalesce(1).write.parquet(s3_bucket, mode="append")